In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import ConvLSTM2D
from matplotlib import pyplot
from numpy import mean
from numpy import std

In [ ]:
prefix = '/content/drive/My Drive/HumanActivityDatasetSplit(3000)'

In [ ]:
def find_longest_df(directory):
    fullfilepaths = list() #list of each files(the full path of file)
    
    filegroups = ['/dws_1','/dws_2','/dws_11','/jog_9','/jog_16','/sit_5','/sit_13',
                  '/std_6','/std_14','/ups_3','/ups_4','/ups_12','/wlk_7','/wlk_8','/wlk_15']
    
    if (directory == prefix + '/train' ):
        namesOfFiles = ['/sub_1.csv','/sub_2.csv','/sub_3.csv','/sub_4.csv','/sub_5.csv',
                        '/sub_6.csv','/sub_7.csv','/sub_8.csv','/sub_9.csv','/sub_10.csv',
                        '/sub_11.csv','/sub_12.csv','/sub_13.csv','/sub_14.csv','/sub_15.csv',
                        '/sub_16.csv','/sub_17.csv']
    else:
         namesOfFiles = ['/sub_18.csv','/sub_19.csv','/sub_20.csv' ,
                         '/sub_21.csv','/sub_22.csv',
                         '/sub_23.csv','/sub_24.csv']
    #loop will put each files full path in a list
    for group in filegroups:
        filepath = directory + group 
        for file in namesOfFiles:
            fullfilepaths.append(filepath + file)
    #return fullfilepaths
    length_of_files = list()
    for file in fullfilepaths:
        df = pd.read_csv(file)
        length_of_files.append(len(df))
    return sorted(length_of_files)[-1]

In [ ]:
#variables to store length of longest files
#longest_file_train = find_longest_df(prefix+'/train')
#longest_file_test = find_longest_df(prefix+'/test')

In [ ]:
#longest_file_test

In [ ]:
#longest_file_train

In [ ]:
#returns file into a numpy array
def load_file(filepath,train=False):
    #if(train==True):
    #    longest_file = longest_file_train
     # else:
    #    longest_file = longest_file_test
    #print('Train is ' + str(train))
    #print('longest file is ' + str(longest_file))
    #dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    dataframe = pd.read_csv(filepath)
    #dataframe = dataframe.drop(labels='Unnamed: 0',axis=1)
    dataframe = dataframe.drop(dataframe.columns[[0]],axis=1)
    ##
    #pad the length of dataframe to be length of file with most timesteps
    #empty_row = pd.DataFrame([[0]*dataframe.shape[1]],columns=dataframe.columns)
    #while(len(dataframe) <longest_file):
    #    dataframe = dataframe.append(empty_row,ignore_index=True)
    ##
    #dataframe = dataframe[:300] #we are only getting first 300 time steps for consistency
    return dataframe.values.reshape(1,len(dataframe),6)

In [ ]:
def load_file_y(filepath):
    dataframe = pd.read_csv(filepath)
    return dataframe.values

In [ ]:
# load a list of files and return as a 3d numpy array
def load_each_file(filepaths, prefix='',train=False):
    #count = 0
    loaded = list()
    for file in filepaths:
        data = load_file(file,train=train)
        #count = count+1
        #print(str(count))
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = np.vstack(loaded)
    return loaded

In [ ]:
def load_dataset_group(directory):
    fullfilepaths = list() #list of each files(the full path of file)
    
    filegroups = ['/dws_1','/dws_2','/dws_11','/jog_9','/jog_16','/sit_5','/sit_13',
                  '/std_6','/std_14','/ups_3','/ups_4','/ups_12','/wlk_7','/wlk_8','/wlk_15']
    
    if (directory == prefix + '/train' ):
        namesOfFiles = ['/sub_1.csv','/sub_2.csv','/sub_3.csv','/sub_4.csv','/sub_5.csv',
                        '/sub_6.csv','/sub_7.csv','/sub_8.csv','/sub_9.csv','/sub_10.csv',
                        '/sub_11.csv','/sub_12.csv','/sub_13.csv','/sub_14.csv','/sub_15.csv',
                        '/sub_16.csv','/sub_17.csv']
        train = True
    else:
        train = False
        namesOfFiles = ['/sub_18.csv','/sub_19.csv','/sub_20.csv' ,
                         '/sub_21.csv','/sub_22.csv',
                         '/sub_23.csv','/sub_24.csv']
    #loop will put each files full path in a list
    for group in filegroups:
        filepath = directory + group 
        for file in namesOfFiles:
            fullfilepaths.append(filepath + file)
            
    X = load_each_file(fullfilepaths,train=train)
    if (directory == prefix + '/train'):
        y = load_file_y(directory + '/y_train.txt')
    else:
        y = load_file_y(directory + '/y_test.txt')
    return X, y 

In [ ]:
#goes into train/test directory
def load_dataset(directory=prefix):
    trainX,trainy = load_dataset_group(directory + '/train')
    testX,testy = load_dataset_group(directory + '/test')
    print(trainX.shape, trainy.shape)
    print(testX.shape, testy.shape)
    
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX,trainy,testX,testy

In [ ]:
'''
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 30, 4
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    #losses = pd.DataFrame(model.history.history)
    #losses.plot()
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy'''

In [ ]:

def evaluate_model(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 0, 50, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape data into time steps of sub-sequences
	n_steps, n_length = 1, 3000
	trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
	# define model
	model = Sequential()
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
	model.add(TimeDistributed(Dropout(0.5)))
	model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
	model.add(TimeDistributed(Flatten()))
	model.add(LSTM(100))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	predictions = model.predict_classes(testX)
	print(predictions)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [ ]:
# fit and evaluate a model
'''
def evaluate_model(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 0, 50, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape into subsequences (samples, time steps, rows, cols, channels)
	n_steps, n_length = 1, 3000
	trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
	# define model
	model = Sequential()
	model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
	model.add(Dropout(0.5))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy'''

In [ ]:
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
 # summarize results
	summarize_results(scores)

In [ ]:
'''def run_experiment(repeats=10):
  #load data
  trainX,trainy,testX,testy = load_dataset()
  scaler = MinMaxScaler()
  for i in range(trainX.shape[0]):
    scaler.fit(trainX[i])
    trainX[i] = scaler.transform(trainX[i])
  for i in range(testX.shape[0]):
    testX[i] = scaler.transform(testX[i])
  #repeat experiment
  scores = list()
  for r in range(repeats):
    score = evaluate_model(trainX,trainy,testX,testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1,score))
    scores.append(score)
  summarize_results(scores)'''

In [ ]:
run_experiment() 